<a href="https://colab.research.google.com/github/submouse9903/uos-deepLearning/blob/main/U47768_CH04_ModelFitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import torchvision

In [ ]:
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 60987744.34it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 93109680.11it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 121685811.25it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 15928535.76it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
#train_dataset[1][0][0,:,:]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [ ]:
plt.imshow(train_dataset[1][0][1,:,:], cmap=plt.get_cmap('gray'))
print( train_dataset[1][0] )

IndexError: ignored

In [ ]:
train_dataset[1][0].shape

torch.Size([1, 28, 28])

In [ ]:
train_dataset[0][0].reshape(-1).shape

torch.Size([784])

In [ ]:
rx = train_dataset.data
ry = train_dataset.targets
rx.shape

torch.Size([60000, 28, 28])

In [ ]:
n = 50000
p = 28*28
x = rx[0:n]
y = ry[0:n]
x = x.reshape(n,-1)
x = x.float()
print("x dtype:", x.dtype)
print("x shape:", x.shape)
print("y dtype:", y.dtype)

x dtype: torch.float32
x shape: torch.Size([50000, 784])
y dtype: torch.int64


In [ ]:
h = 64
output_dim = 10

In [ ]:
model = nn.Sequential(nn.Linear(p,h),
                      nn.ReLU(),
                      nn.Linear(h, h),
                      nn.ReLU(),
                      nn.Linear(h, h),
                      nn.ReLU(),
                      nn.Linear(h, output_dim),
                      nn.Sigmoid(),
                      nn.Softmax())

In [ ]:
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=10, bias=True)
  (7): Sigmoid()
  (8): Softmax(dim=None)
)


In [ ]:
learning_rate = 0.005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
cross_entropy = nn.CrossEntropyLoss()  

In [ ]:
output = model(x)
output[0,:]

tensor([0.1299, 0.1433, 0.0926, 0.0575, 0.1390, 0.0888, 0.1494, 0.0720, 0.0667,
        0.0607], grad_fn=<SliceBackward0>)

In [ ]:
output.shape

torch.Size([50000, 10])

In [ ]:
num_B = 300
loss_vec = np.zeros(num_B)
optimizer.zero_grad()
for i in range(num_B):
  output = model(x)
  loss = cross_entropy(output, y)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  predicted = torch.max(output.data,1)
  v =((predicted.indices == y).sum()/n ).item()
  loss_vec[i] = v
  if i%30 == 0:
    print ("Accuracy: {:.4f}".format(v))

Accuracy: 0.1335
Accuracy: 0.8354
Accuracy: 0.8640


KeyboardInterrupt: ignored

In [ ]:
n = 10000
p = 28*28
x_test = rx[50000:60000]
y_test = ry[50000:60000]
x_test = x_test.reshape(n,-1)
x_test = x_test.float()
print("x dtype:", x_test.dtype)
print("x shape:", x_test.shape)
print("y dtype:", y_test.dtype)

x dtype: torch.float32
x shape: torch.Size([10000, 784])
y dtype: torch.int64


In [ ]:
output = model(x_test)
predicted = torch.max(output.data,1)
v =((predicted.indices == y_test).sum()/n ).item()
v

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.9544000029563904